In [1]:
import os
import numpy as np
import pandas as pd

read_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_region = 'Region'
col_year = 'Year'
col_gdp = 'GDP'

In [2]:
df = pd.read_csv(read_path + 'GoldDataframe.csv', index_col = [col_country, col_region, col_year])
df

AgriShareGDP  CreditToAgriFishForest  \
Country     Region             Year                                         
Afghanistan South Asia         2000      54.06300                     NaN   
                               2001      54.06300                     NaN   
                               2002      45.13440                     NaN   
                               2003      41.90340                     NaN   
                               2004      35.61280                     NaN   
...                                           ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016       7.87399                     NaN   
                               2017       8.34095                     NaN   
                               2018       8.30469                     NaN   
                               2019       8.17322                     NaN   
                               2020      10.93630                     NaN   

                                     EmploymentRural           GDP  \
Country     Region             Year                                  
Afghanistan South Asia         2000              NaN   3342.034168   
                               2001              NaN   3598.470576   
                               2002              NaN   4141.523943   
                               2003              NaN   4729.042179   
                               2004              NaN   5388.482107   
...                                              ...           ...   
Zimbabwe    Sub-Saharan Africa 2016              NaN  20548.678073   
                               2017              NaN  22040.902301   
                               2018              NaN  24311.560545   
                               2019              NaN  21935.075306   
                               2020              NaN  21786.743637   

                                     % Soldiers  Employment in industry  \
Country     Region             Year                                       
Afghanistan South Asia         2000    7.887961                    9.48   
                               2001    5.020511                    8.98   
                               2002    2.153062                    9.99   
                               2003    2.208290                   10.35   
                               2004    0.435599                   10.61   
...                                         ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016    0.759750                    7.05   
                               2017    0.750720                    6.90   
                               2018    0.738210                    6.75   
                               2019    0.738210                    6.57   
                               2020    0.738210                    6.57   

                                     Employment in services  Birth Rate  \
Country     Region             Year                                       
Afghanistan South Asia         2000               24.680000      48.021   
                               2001               24.719999      47.505   
                               2002               25.590000      46.901   
                               2003               25.950001      46.231   
                               2004               26.120001      45.507   
...                                                     ...         ...   
Zimbabwe    Sub-Saharan Africa 2016               26.070000      32.864   
                               2017               26.629999      31.732   
                               2018               27.230000      30.676   
                               2019               27.240000      29.747   
                               2020               27.240000      28.977   

                                     Cost business start-up  Death Rate  ...  \
Country     Region             Year                                      ...   
Afghanistan 

In [47]:
country_list = list(set(df.index.get_level_values(col_country)))
region_list = list(set(df.index.get_level_values(col_region)))

country_list.sort()
region_list.sort()

shift_list = range(0, 21)  # GENERALIZAR PARA CUALQUIER RANGO DE AÑOS

for country in country_list:
    corr_list = []
    
    #Get the DataFrame for a given country.
    country_df = df.loc[df.index.get_level_values(col_country) == country]
    min_year = country_df.index.get_level_values(col_year).min()
    max_year = country_df.index.get_level_values(col_year).max()

    gdp_series = country_df.loc[:, col_gdp]
    ind_df = country_df.drop(col_gdp, axis = 1).dropna(axis = 1)

    #corr_df = ind_df.corrwith(gdp_series, method = 'spearman')
    #display(corr_df)

    for shift in [12, 13, 14]:
        shifted_min = min_year + shift
        shifted_max = max_year - shift

        # Effect of indicators on the GDP.
        shifted_gdp = gdp_series.loc[gdp_series.index.get_level_values(col_year) >= shifted_min]
        shifted_ind = ind_df.loc[ind_df.index.get_level_values(col_year) <= shifted_max]

        #display(shifted_ind.index.get_level_values(col_year))
        shifted_ind.index = shifted_ind.index.map(lambda x: (x[0], x[1], x[2] + shift))           #MEJORABLEEEEEEE
        

        #display(shifted_gdp)
        #display(shifted_ind)
        #display(ind_df.loc[ind_df.index.get_level_values(col_year) < shifted_max].apply(lambda x: x.index.get_level_values(col_year) + shift))

        country_corr = shifted_ind.corrwith(shifted_gdp, method = 'spearman')

        country_corr.name = shift
        #display(country_corr)
        #country_corr.rename
        #display(country_corr_df)
        #country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_gdp])

        #country_corr_series = country_corr_df.loc[:, country]
        corr_list.append(country_corr)

    corr_df = pd.concat(corr_list, axis = 1).transpose().sort_index()
    corr_df.index = pd.MultiIndex.from_tuples((country, sh) for sh, _ in corr_df.iterrows())
    corr_df.index.names = [col_country, 'Shift']

    #print(corr_df.loc[:, 'Population'].max())

    #val = corr_df.apply.groupby(level = col_country)(lambda x: max(x.min(), x.max(), key=abs))
    #display(corr_df)
    val = corr_df.apply(lambda x: max(x.min(), x.max(), key=abs))

    
    #.apply(lambda x: max(x.min(), x.max(), key=abs))
    #display(val)
    val.name = country

    max_row = pd.DataFrame(val)
    display(max_row.transpose())
    display(corr_df)

    max_dict = {}



    """ for ind in corr_df.columns:


        max = corr_df[ind].max()
        min = corr_df[ind].min()
        val = max if abs(max) > abs(min) else min
        index = corr_df.loc[corr_df[ind] == val].index.values
        print(index)
        shift = index[len(country):]
        
        max_dict[str(ind) + ' (+ ' + str(shift) + ')'] = val """

    max_corr = pd.DataFrame(max_dict)
    display(max_corr)

    break
        #Correlation matrix for that country, using Spearman.
        #country_corr_df = country_df.corr(method = 'spearman')
    """ except:
        print(country) """

Int64Index([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008], dtype='int64', name='Year')

AgriShareGDP                      0.184102
% Soldiers                        0.500000
Employment in industry           -0.133333
Employment in services           -0.100000
Birth Rate                        0.100000
Cost business start-up           -0.182574
Death Rate                        0.100000
Employment in agriculture         0.133333
% Female Employment              -0.750000
% Male Employment                 0.326363
Life Expectancy                  -0.100000
Maternal Death Risk               0.100000
Infant Mortality                  0.100000
% Population Growth              -0.366667
% Rural Population                0.100000
% Vulnerable female employment    0.133333
% Vulnerable male employment      0.133333
Civil Liberties                   0.112640
Freedom of Expression            -0.127688
% Healthcare Investment          -0.610257
Population                       -0.100000
Name: 12, dtype: float64

Int64Index([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007], dtype='int64', name='Year')

AgriShareGDP                      0.179644
% Soldiers                        0.261905
Employment in industry            0.047619
Employment in services            0.023810
Birth Rate                       -0.023810
Cost business start-up            0.136386
Death Rate                       -0.023810
Employment in agriculture        -0.047619
% Female Employment              -0.547619
% Male Employment                 0.682647
Life Expectancy                   0.023810
Maternal Death Risk              -0.023810
Infant Mortality                 -0.023810
% Population Growth              -0.809524
% Rural Population               -0.023810
% Vulnerable female employment   -0.047619
% Vulnerable male employment     -0.047619
Civil Liberties                  -0.743330
Freedom of Expression             0.049103
% Healthcare Investment          -0.609938
Population                        0.023810
Name: 13, dtype: float64

Int64Index([2000, 2001, 2002, 2003, 2004, 2005, 2006], dtype='int64', name='Year')

AgriShareGDP                     -0.342356
% Soldiers                        0.071429
Employment in industry            0.392857
Employment in services            0.285714
Birth Rate                       -0.285714
Cost business start-up           -0.267261
Death Rate                       -0.285714
Employment in agriculture        -0.392857
% Female Employment               0.035714
% Male Employment                 0.270281
Life Expectancy                   0.285714
Maternal Death Risk              -0.285714
Infant Mortality                 -0.285714
% Population Growth              -0.750000
% Rural Population               -0.285714
% Vulnerable female employment   -0.392857
% Vulnerable male employment     -0.392857
Civil Liberties                  -0.347183
Freedom of Expression             0.467801
% Healthcare Investment           0.037062
Population                        0.285714
Name: 14, dtype: float64

AgriShareGDP  % Soldiers  Employment in industry  \
Country     Shift                                                     
Afghanistan 12         0.184102    0.500000               -0.133333   
            13         0.179644    0.261905                0.047619   
            14        -0.342356    0.071429                0.392857   

                   Employment in services  Birth Rate  Cost business start-up  \
Country     Shift                                                               
Afghanistan 12                  -0.100000    0.100000               -0.182574   
            13                   0.023810   -0.023810                0.136386   
            14                   0.285714   -0.285714               -0.267261   

                   Death Rate  Employment in agriculture  % Female Employment  \
Country     Shift                                                               
Afghanistan 12       0.100000                   0.133333            -0.750000   
            13      -0.023810                  -0.047619            -0.547619   
            14      -0.285714                  -0.392857             0.035714   

                   % Male Employment  ...  Maternal Death Risk  \
Country     Shift                     ...                        
Afghanistan 12              0.326363  ...             0.100000   
            13              0.682647  ...            -0.023810   
            14              0.270281  ...            -0.285714   

                   Infant Mortality  % Population Growth  % Rural Population  \
Country     Shift                                                              
Afghanistan 12             0.100000            -0.366667            0.100000   
            13            -0.023810            -0.809524           -0.023810   
            14            -0.285714            -0.750000           -0.285714   

                   % Vulnerable female employment  \
Country     Shift                                   
Afghanistan 12                           0.133333   
            13                          -0.047619   
            14                          -0.392857   

                   % Vulnerable male employment  Civil Liberties  \
Country     Shift                                                  
Afghanistan 12                         0.133333         0.112640   
            13                        -0.047619        -0.743330   
            14                        -0.392857        -0.347183   

                   Freedom of Expression  % Healthcare Investment  Population  
Country     Shift                                                              
Afghanistan 12                 -0.127688                -0.610257   -0.100000  
            13                  0.049103                -0.609938    0.023810  
            14                  0.467801                 0.037062    0.285714  

[3 rows x 21 columns]

""


In [4]:
df1 = pd.DataFrame({1: [9, 3, 8], 2: [1, 5, 5], 3: [2, 2, 1]})
df1

df2 = pd.DataFrame({'t': ['a', 'a', 'b'], 1: [100, 388, 300], 2: [400, 124, 600], 3: [700, 977, 900]})
df2

df1.corrwith(df2)

display(df2)
dfs = df2.copy().loc[:, 1]
display(dfs)

df1.corrwith(dfs)
df2.groupby(['t']).sum()


,t,1,2,3
0,a,100,400,700
1,a,388,124,977
2,b,300,600,900


0    100
1    388
2    300
Name: 1, dtype: int64

,1,2,3
t,,,
a,488,524,1677
b,300,600,900
